In [7]:
from scipy.spatial.distance import pdist
from scipy.spatial.distance import cdist
import numpy as np
import pandas as pd
% matplotlib inline
import matplotlib.pyplot as plt
import scipy.linalg as lng
import matplotlib
from numpy import genfromtxt
import cvxopt
from math import *

In [2]:
my_data = genfromtxt('data/Xtr.csv', delimiter=',')
x_tr = my_data[:,0:-1]

In [3]:
def img_rescale(img):
    min_value = np.min(img)
    result = 255*(img-min_value)/np.max(img-min_value)
    return np.cast[np.uint8](result)

def convolve(x,kernel):
    y = np.zeros(x.shape)
    #print(kernel.shape)
    kRows,kCols = kernel.shape
    rows,cols = x.shape
    kCenterX = floor(kCols / 2)
    kCenterY = floor(kRows / 2)
    #print(kCenterX,kCenterY)
    for i in range(rows):             
        for j in range(cols):      
            for m in range(kRows):  
                mm = kRows - 1 - m
                for n in range(kCols):
                    nn = kCols - 1 - n
                    ii = i + (m - kCenterY)
                    jj = j + (n - kCenterX)
                    if( ii >= 0 and ii < rows and jj >= 0 and jj < cols ):
                        y[i][j] += x[ii][jj] * kernel[mm][nn]
    return y

In [4]:
#a tester avec une echelle hsv au lieu de rgb
def intensity_histogram(x):
    w,h,d=x.shape
    result = np.zeros((16,16,16))
    x_rescale = x/16
    for i in range(w):
        for j in range(h):
                temp = x_rescale[i,j,:]
                result[temp[0]][temp[1]][temp[2]]=  result[temp[0]][temp[1]][temp[2]]+1
    return result

def trans_histogram(x):
    result_h = np.zeros((16,16,16))
    result_v = np.zeros((16,16,16))
    diff_h =x[:,1:,:]
    diff_v =x[1:,:,:]
    trans_v = np.absolute(x[0:-1,:,:]-diff_v)
    trans_h = np.absolute(x[:,0:-1,:]-diff_h)
    result_h = intensity_histogram(trans_h)
    result_v = intensity_histogram(trans_v)
    return result_v, result_h

In [5]:
# sift features
Nangles = 8
Nbins = 4
Nsamples = Nbins**2
alpha = 9.0
angles = np.array(range(Nangles))*2.0*np.pi/Nangles

def gen_dgauss(sigma):
    '''
    generating a derivative of Gauss filter on both the X and Y
    direction.
    '''
    fwid = np.int(2*np.ceil(sigma))
    G = np.array(range(-fwid,fwid+1))**2
    G = G.reshape((G.size,1)) + G
    G = np.exp(- G / 2.0 / sigma / sigma)
    G /= np.sum(G)
    GH,GW = np.gradient(G)
    GH *= 2.0/np.sum(np.abs(GH))
    GW *= 2.0/np.sum(np.abs(GW))
    return GH,GW

class DsiftExtractor:
    '''
    The class that does dense sift feature extractor.
    Sample Usage:
        extractor = DsiftExtractor(gridSpacing,patchSize,[optional params])
        feaArr,positions = extractor.process_image(Image)
    '''
    def __init__(self, gridSpacing, patchSize,
                 nrml_thres = 1.0,\
                 sigma_edge = 0.8,\
                 sift_thres = 0.2):
        '''
        gridSpacing: the spacing for sampling dense descriptors
        patchSize: the size for each sift patch
        nrml_thres: low contrast normalization threshold
        sigma_edge: the standard deviation for the gaussian smoothing
            before computing the gradient
        sift_thres: sift thresholding (0.2 works well based on
            Lowe's SIFT paper)
        '''
        self.gS = gridSpacing
        self.pS = patchSize
        self.nrml_thres = nrml_thres
        self.sigma = sigma_edge
        self.sift_thres = sift_thres
        # compute the weight contribution map
        sample_res = self.pS / np.double(Nbins)
        sample_p = np.array(range(self.pS))
        sample_ph, sample_pw = np.meshgrid(sample_p,sample_p)
        sample_ph.resize(sample_ph.size)
        sample_pw.resize(sample_pw.size)
        bincenter = np.array(range(1,Nbins*2,2)) / 2.0 / Nbins * self.pS - 0.5 
        bincenter_h, bincenter_w = np.meshgrid(bincenter,bincenter)
        bincenter_h.resize((bincenter_h.size,1))
        bincenter_w.resize((bincenter_w.size,1))
        dist_ph = abs(sample_ph - bincenter_h)
        dist_pw = abs(sample_pw - bincenter_w)
        weights_h = dist_ph / sample_res
        weights_w = dist_pw / sample_res
        weights_h = (1-weights_h) * (weights_h <= 1)
        weights_w = (1-weights_w) * (weights_w <= 1)
        # weights is the contribution of each pixel to the corresponding bin center
        self.weights = weights_h * weights_w
        #pyplot.imshow(self.weights)
        #pyplot.show()
        
    def process_image(self, image, positionNormalize = True,\
                       verbose = True):
        '''
        processes a single image, return the locations
        and the values of detected SIFT features.
        image: a M*N image which is a numpy 2D array. If you 
            pass a color image, it will automatically be converted
            to a grayscale image.
        positionNormalize: whether to normalize the positions
            to [0,1]. If False, the pixel-based positions of the
            top-right position of the patches is returned.
        
        Return values:
        feaArr: the feature array, each row is a feature
        positions: the positions of the features
        '''

        image = image.astype(np.double)
        if image.ndim == 3:
            # we do not deal with color images.
            image = np.mean(image,axis=2)
        # compute the grids
        H,W = image.shape
        gS = self.gS
        pS = self.pS
        remH = np.mod(H-pS, gS)
        remW = np.mod(W-pS, gS)
        offsetH = remH/2
        offsetW = remW/2
        #print(type(offsetH),type(H-pS+1),type(gS))
        gridH,gridW = np.meshgrid(range(int(offsetH),H-pS+1,gS), range(int(offsetW),W-pS+1,gS))
        gridH = gridH.flatten()
        gridW = gridW.flatten()
        #if verbose:
            #print('Image: w {}, h {}, gs {}, ps {}, nFea {}'.\
                    #format(W,H,gS,pS,gridH.size))
        feaArr = self.calculate_sift_grid(image,gridH,gridW)
        feaArr = self.normalize_sift(feaArr)
        if positionNormalize:
            positions = np.vstack((gridH / np.double(H), gridW / np.double(W)))
        else:
            positions = np.vstack((gridH, gridW))
        return feaArr, positions

    def calculate_sift_grid(self,image,gridH,gridW):
        '''
        This function calculates the unnormalized sift features
        It is called by process_image().
        '''
        H,W = image.shape
        Npatches = gridH.size
        feaArr = np.zeros((Npatches,Nsamples*Nangles))

        # calculate gradient
        GH,GW = gen_dgauss(self.sigma)
        IH = convolve(image,GH)
        IW = convolve(image,GW)
        Imag = np.sqrt(IH**2+IW**2)
        Itheta = np.arctan2(IH,IW)
        Iorient = np.zeros((Nangles,H,W))
        for i in range(Nangles):
            Iorient[i] = Imag * np.maximum(np.cos(Itheta - angles[i])**alpha,0)
            #pyplot.imshow(Iorient[i])
            #pyplot.show()
        for i in range(Npatches):
            currFeature = np.zeros((Nangles,Nsamples))
            for j in range(Nangles):
                currFeature[j] = np.dot(self.weights,\
                        Iorient[j,gridH[i]:gridH[i]+self.pS, gridW[i]:gridW[i]+self.pS].flatten())
            feaArr[i] = currFeature.flatten()
        return feaArr

    def normalize_sift(self,feaArr):
        '''
        This function does sift feature normalization
        following David Lowe's definition (normalize length ->
        thresholding at 0.2 -> renormalize length)
        '''
        siftlen = np.sqrt(np.sum(feaArr**2,axis=1))
        hcontrast = (siftlen >= self.nrml_thres)
        siftlen[siftlen < self.nrml_thres] = self.nrml_thres
        # normalize with contrast thresholding
        feaArr /= siftlen.reshape((siftlen.size,1))
        # suppress large gradients
        feaArr[feaArr>self.sift_thres] = self.sift_thres
        # renormalize high-contrast ones
        feaArr[hcontrast] /= np.sqrt(np.sum(feaArr[hcontrast]**2,axis=1)).\
                reshape((feaArr[hcontrast].shape[0],1))
        return feaArr

class SingleSiftExtractor(DsiftExtractor):
    '''
    The simple wrapper class that does feature extraction, treating
    the whole image as a local image patch.
    '''
    def __init__(self, patchSize,
                 nrml_thres = 1.0,\
                 sigma_edge = 0.8,\
                 sift_thres = 0.2):
        # simply call the super class __init__ with a large gridSpace
        DsiftExtractor.__init__(self, patchSize, patchSize, nrml_thres, sigma_edge, sift_thres)   
    
    def process_image(self, image):
        return DsiftExtractor.process_image(self, image, False, False)[0]


In [8]:
extractor = DsiftExtractor(8,16,1)
sift_hist= np.zeros((x_tr.shape[0],9,128))
for i in range(x_tr.shape[0]):
    #print(i)
    im =x_tr[i,:]
    im1=np.zeros((32,32,3))
    for k in range(3):
        im1[:,:,k]=im[k*1024:k*1024+1024].reshape((32,32))
    im1= img_rescale(im1)
    image = np.mean(np.double(im1),axis=2)
    feaArr,positions = extractor.process_image(image)
    sift_hist[i,:,:]=feaArr

In [9]:
y_data = pd.read_csv('data/Ytr.csv', sep = ',')
y = y_data['Prediction'].values
X=sift_hist.reshape((x_tr.shape[0],9*128))

In [9]:
#np.savetxt('../X_sift.txt',X)

In [10]:
y_modified_train = []
y_modified_test = []
x_modified_train = []
x_modified_test = []
for i in range(0,10):
    y_class = y.copy()
    y_temp = y_class[y_class == i]
    x_temp = X[y_class == i]
    indices = np.random.permutation(np.arange(len(y_temp)))
    full_train = indices
    y_modified_train.append(y_temp[full_train])
    x_modified_train.append(x_temp[full_train,:])

In [11]:
X_train = np.vstack(x_modified_train)
Y_train = np.hstack(y_modified_train)
indices = np.random.permutation(np.arange(len(Y_train)))
X_train = X_train[indices,:]
Y_train = Y_train[indices]

In [12]:
list_y_train = []
for i in range(0,10):
    y_class = Y_train.copy()
    y_class[Y_train != i] = -1
    y_class[Y_train == i] = 1
    list_y_train.append(y_class)

The structure of the SVM is inspired from http://tullo.ch/articles/svm-py/

In [13]:
class SVMPredictor(object):
    def __init__(self,gamma,bias,weights,support_vectors,support_vector_labels):
        self.gamma = gamma 
        self._bias = bias
        self._weights = weights
        self._support_vectors = support_vectors
        self._support_vector_labels = support_vector_labels


 

    def predictest(self,X):
        K = -self.gamma * cdist(self._support_vectors,X ,'cityblock')
        np.exp(K, K)
        taille = len(self._support_vector_labels)
        lab = self._support_vector_labels.reshape((taille,1))
        wei = self._weights.reshape((taille,1))
        result = np.sum(np.multiply(wei,np.multiply(lab,K)),axis = 0)
        return np.sign(result),result

import numpy as np
import numpy.linalg as la




In [14]:
#from sklearn.metrics.pairwise import manhattan_distances
MIN_SUPPORT_VECTOR_MULTIPLIER = 1e-5
class SVMTrainer(object):
    def __init__(self, C=1,C1=1,gamma = 1,option = "None"):
        self.C = C
        self.C1 = C1
        self.gamma = gamma   ###kernel laplacian
        self.option = option
    def train(self, X, y):
        """Given the training features X with labels y, returns a SVM
        predictor representing the trained SVM.
        """
        lagrange_multipliers = self._compute_multipliers(X, y)
        return self._construct_predictor(X, y, lagrange_multipliers)

    def _gram_matrix(self, X):
        gamma = self.gamma
        K = -gamma * cdist(X,X ,'cityblock')
        np.exp(K, K)
        
        return K

    def _construct_predictor(self, X, y, lagrange_multipliers):
        support_vector_indices = lagrange_multipliers > MIN_SUPPORT_VECTOR_MULTIPLIER

        support_multipliers = lagrange_multipliers[support_vector_indices]
        support_vectors = X[support_vector_indices]
        support_vector_labels = y[support_vector_indices]
        a=SVMPredictor(gamma=self.gamma,bias=0.0,weights=support_multipliers,
             support_vectors=support_vectors,support_vector_labels=support_vector_labels).predictest(support_vectors)[0]
        print(support_vector_labels.shape)
        bias = np.mean(support_vector_labels-a)
        return SVMPredictor(gamma=self.gamma,bias=bias,weights=support_multipliers,support_vectors=support_vectors,
            support_vector_labels=support_vector_labels)
      
    def _compute_multipliers(self, X, y):
        n_samples, n_features = X.shape

        K = self._gram_matrix(X)


        P = cvxopt.matrix(np.outer(y, y) * K)
        q = cvxopt.matrix(-1 * np.ones(n_samples))

        G_std = cvxopt.matrix(np.diag(np.ones(n_samples) * -1))
        h_std = cvxopt.matrix(np.zeros(n_samples))
        G_slack = cvxopt.matrix(np.diag(np.ones(n_samples)))
        
        if(self.option == "BS"):
            tmp2 = np.sign(y)
            h_slack = np.sign(y)
            h_slack[tmp2 == 1] = self.C
            h_slack[tmp2 == -1] = self.C1
            h_slack = cvxopt.matrix(h_slack)
        else:
            h_slack = cvxopt.matrix(np.ones(n_samples) * self._C)
         
        
        G = cvxopt.matrix(np.vstack((G_std, G_slack)))
        h = cvxopt.matrix(np.vstack((h_std, h_slack)))

        A = cvxopt.matrix(y, (1, n_samples))
        b = cvxopt.matrix(0.0)
        solution = cvxopt.solvers.qp(P, q, G, h, A, b)

        return np.ravel(solution['x'])

In [16]:
from itertools import product   ###one vs one 
s = [tuple(sorted((i,j))) for i,j in product(range(0,10),range(0,10))]
indices = []
SVMT3 = []
list_SVM3 = []
for i,j in sorted(list(set(s))):
    if(i!=j):
        print(i,j)
        indices.append((i,j))
        temp = Y_train.copy()
        print(temp.shape)
        x_pos = X_train[temp == i]
        x_neg = X_train[temp == j]
        temp_pos = temp[temp == i]
        temp_neg = temp[temp == j]
        temp_neg = -1*np.ones((temp_neg.shape))
        temp_pos = np.ones((temp_pos.shape))
        temp_train = np.vstack((x_pos,x_neg))
        temp_y = np.hstack((temp_pos,temp_neg))
        permut = np.random.permutation(len(temp_train))
        temp_train = temp_train[permut,:]
        temp_y = temp_y[permut]
        n_samples = len(temp_train)
        C = 0.1  ### remplacer par 2 
        C1 = 0.1 ### remplacer par 2
        #print(C,C1)
        svm = (SVMTrainer(C = C,C1 = C1 , option = "BS",
                           gamma =(1.0/float(X_train.shape[1]) ))) ### ajuster parametres !!!
        SVMT3.append(svm)
        list_SVM3.append(svm.train(temp_train, temp_y))


0 1
(5000,)
     pcost       dcost       gap    pres   dres
 0: -3.8602e+02 -2.4912e+02  5e+03  2e+01  8e-15
 1: -9.9881e+01 -2.1556e+02  3e+02  9e-01  7e-15
 2: -8.3592e+01 -1.2506e+02  4e+01  8e-16  2e-15
 3: -9.5245e+01 -9.6617e+01  1e+00  2e-16  2e-15
 4: -9.6221e+01 -9.6234e+01  1e-02  3e-15  2e-15
 5: -9.6230e+01 -9.6231e+01  1e-04  9e-16  2e-15
 6: -9.6231e+01 -9.6231e+01  1e-06  3e-15  2e-15
Optimal solution found.
(1000,)
0 2
(5000,)
     pcost       dcost       gap    pres   dres
 0: -3.9997e+02 -2.5096e+02  6e+03  2e+01  8e-15
 1: -9.8510e+01 -2.2078e+02  3e+02  9e-01  7e-15
 2: -8.3080e+01 -1.2619e+02  4e+01  4e-16  2e-15
 3: -9.5262e+01 -9.6753e+01  1e+00  2e-16  2e-15
 4: -9.6326e+01 -9.6341e+01  2e-02  2e-15  2e-15
 5: -9.6337e+01 -9.6337e+01  2e-04  3e-16  2e-15
 6: -9.6337e+01 -9.6337e+01  2e-06  2e-15  2e-15
Optimal solution found.
(1000,)
0 3
(5000,)
     pcost       dcost       gap    pres   dres
 0: -3.4958e+02 -2.5719e+02  6e+03  2e+01  9e-15
 1: -1.0153e+02 -2.23

#### preparation test set sift

In [18]:
my_data_test = genfromtxt('data/Xte.csv', delimiter=',')
x_te = my_data_test[:,0:-1]
extractor = DsiftExtractor(8,16,1)
sift_hist= np.zeros((x_te.shape[0],9,128))
for i in range(x_te.shape[0]):
    #print(i)
    im =x_te[i,:]
    im1=np.zeros((32,32,3))
    for k in range(3):
        im1[:,:,k]=im[k*1024:k*1024+1024].reshape((32,32))
    im1= img_rescale(im1)
    image = np.mean(np.double(im1),axis=2)
    feaArr,positions = extractor.process_image(image)
    sift_hist[i,:,:]=feaArr
    #X=sift_hist.reshape((x_tr.shape[0],9*128))

In [19]:
X_te=sift_hist.reshape((x_te.shape[0],9*128))

In [20]:
#np.savetxt('../X_sift_test.txt',X_te)

In [21]:
score = np.zeros((len(X_te),len(indices)))
for j in range(len(indices)) :
    score[:,j] = list_SVM3[j].predictest(X_te)[0]
new_score = np.zeros(score.shape)
for i in range(0,len(X_te)) :
    for j in range(len(indices)) :
        if(score[i,j]<0):
            new_score[i,j] = indices[j][1]
        else:
            new_score[i,j] = indices[j][0]
new_score = new_score.astype('int')

tes = np.argmax(np.vstack([np.bincount(new_score[i,:],minlength=10) for i in range(new_score.shape[0])]),axis = 1)

In [22]:
df_submit = pd.DataFrame()

In [23]:
df_submit['Id'] = np.arange(1,2001)
df_submit['Prediction'] = tes
df_submit.tail()
df_submit.to_csv('prediction_3.csv',index = None)